In [ ]:
import json
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
from preprocess import MediumBlogPost

import seaborn.apionly as sns
import matplotlib.pyplot as plt
%matplotlib inline


file_path = '/Users/Kevin/Downloads/data-science.jl'

In [ ]:
dfs = []
broadcast_dfs = []
with open(file_path) as f:
    for line in f:
        # get raw data
        row = MediumBlogPost(**json.loads(line)).to_frame()
        dfs.append(row)
        # get df_broadcast
        row_b = pd.DataFrame(MediumBlogPost(**json.loads(line)).to_dict())
        broadcast_dfs.append(row_b)


df = pd.concat(dfs).reset_index(drop=True)
df_broadcast = pd.concat(broadcast_dfs).reset_index(drop=True)